
PROYECTO INTELIGENCIA ARTIFICIAL

PONTIFICIA UNIVERSIDAD JAVERIANA

JUAN PABLO GARCIA R.

OBJETIVO: Utilizar distintos métodos de ML para clasificar si una persona padece de una enfermedad cardíaca o no, a partir de datos tomados al llegar a la UCI.

In [16]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sb
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import LogisticRegression



datos = pd.read_csv('heart.csv')
print(datos.info())
datos.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB
None


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


### División del conjunto de datos

In [58]:
datos_1=datos.copy()
np.random.seed(0)

X=datos_1.drop(["target",], axis=1)  #Caracteristicas
Y=pd.DataFrame(datos_1['target'])           #Etiquetas

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42,test_size=0.2)

### Regresion Logística

In [20]:

from scipy.spatial import distance# por favor buscar la documentacion oficial de la función https://docs.scipy.org/doc/scipy/reference/spatial.distance.html#module-scipy.spatial.distance
from sklearn.preprocessing import StandardScaler# por favor buscar la documentacion oficial de la función
#https://scikit-learn.org/stable/modules/preprocessing.html
%matplotlib inline
#https://stackoverflow.com/questions/43027980/purpose-of-matplotlib-inline
from sklearn.datasets import make_blobs
#https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html
from sklearn.datasets import make_circles

# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve,roc_auc_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [70]:
##Vectores para almacenar los hiperparámetros usados, y sus correspondientes puntajes

MCC_list=[]
F1_list=[]
AUC_list=[]

scaler_list=[]
solver_list=[]
C_list=[]

for j in range(0,3):                    ##ITERACIÓN DEL ESCALIZADOR
  
  if j==0:
    scaler = MinMaxScaler()
    sca='MinMaxScaler()'
  elif j==1:
    scaler = RobustScaler()
    sca ='RobustScaler()'
  elif j==2:
    scaler = StandardScaler()
    sca='StandardScaler()'


  
  scaler.fit(X_train)
  X_train_1 = scaler.transform(X_train)
  X_test_1 = scaler.transform(X_test)

  pca = PCA(n_components=13)          ##13 COMPONENTES, LA DIMENSIÓN QUE MENOS VARIANZA EXPLICA, EXPLICA EL 2.6%, LAS DEJO TODAS
  pca.fit(X_train_1)
  X_train_1 = pca.transform(X_train_1)
  X_test_1 = pca.transform(X_test_1)
  print("Pesos de PCA:",pca.explained_variance_ratio_)
  #print("Componentes de PCA:",pca.components_)

  



  solvs=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
  for solver in range(0,5):              ##ITERACIÓN DEL SOLVER
    for C in [0.01,0.05,0.1,0.5,1,5,10,50,100,500]:                        ##ITERACIÓN DEL C, variacion logaritmica

            print('Escalizador =',sca,'solver=',solvs[solver],',','C=',C,)
            ##Almacenar los hiperparámetros en las listas
            scaler_list.append(scaler)
            solver_list.append(solvs[solver])
            C_list.append(C)
            

            ##Aplicar el modelo
            LR=LogisticRegression(penalty='l2',solver=solvs[solver], C=C)
            LR.fit(X_train_1, y_train)
            y_test_predicted = LR.predict(X_test_1)
            
            proba=LR.predict_proba(X_test_1)##prob1
            y_test_scores = proba[:,1:].flatten()##prob2

            MCC = matthews_corrcoef(y_test, y_test_predicted)
            print("matthews_corrcoef", MCC)
            F1 = f1_score(y_test, y_test_predicted, average='micro')
            print("f1 =", F1)
            roc_auc=metrics.roc_auc_score(y_test, y_test_scores)
            print("roc_auc =", roc_auc)

            #Almacenar los scores en las listas
            MCC_list.append(MCC)
            F1_list.append(F1)
            AUC_list.append(roc_auc)
      

maxMCC=np.argmax(MCC_list)
maxF1=np.argmax(F1_list)
maxAUC=np.argmax(AUC_list)

print("Valor del mejor MCC = ",MCC_list[maxMCC])
print("Valor del mejor F1 = ",F1_list[maxF1])
print("Valor del mejor AUC_roc = ",AUC_list[maxAUC])

print('Parametros mejor MCC:','penalty = l2,','Escalizador =',scaler_list[maxMCC],', solver=',solver_list[maxMCC],',','C=',C_list[maxMCC])
print('Parametros mejor F1:','penalty = l2,','Escalizador =',scaler_list[maxF1],', solver=',solver_list[maxF1],',','C=',C_list[maxF1])
print('Parametros mejor AUC_roc:','penalty = l2,','Escalizador =',scaler_list[maxAUC],', solver=',solver_list[maxAUC],',','C=',C_list[maxAUC])

Pesos de PCA: [0.27173097 0.19243203 0.11711389 0.08507425 0.08130784 0.06387459
 0.05517117 0.03314911 0.03074089 0.02484146 0.01832923 0.01605866
 0.0101759 ]
Escalizador = MinMaxScaler() solver= newton-cg , C= 0.01
matthews_corrcoef 0.47711642835569656
f1 = 0.7213114754098362
roc_auc = 0.8987068965517242
Escalizador = MinMaxScaler() solver= newton-cg , C= 0.05
matthews_corrcoef 0.7370689655172413
f1 = 0.8688524590163934
roc_auc = 0.9030172413793104
Escalizador = MinMaxScaler() solver= newton-cg , C= 0.1
matthews_corrcoef 0.709051724137931
f1 = 0.8524590163934426
roc_auc = 0.908405172413793
Escalizador = MinMaxScaler() solver= newton-cg , C= 0.5
matthews_corrcoef 0.7395052723482932
f1 = 0.8688524590163934
roc_auc = 0.9213362068965517
Escalizador = MinMaxScaler() solver= newton-cg , C= 1
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.9245689655172414
Escalizador = MinMaxScaler() solver= newton-cg , C= 5
matthews_corrcoef 0.6738432321543399
f1 = 0.836065573770

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

matthews_corrcoef 0.7370689655172413
f1 = 0.8688524590163934
roc_auc = 0.9030172413793104
Escalizador = MinMaxScaler() solver= lbfgs , C= 0.1
matthews_corrcoef 0.709051724137931
f1 = 0.8524590163934426
roc_auc = 0.908405172413793
Escalizador = MinMaxScaler() solver= lbfgs , C= 0.5
matthews_corrcoef 0.7395052723482932
f1 = 0.8688524590163934
roc_auc = 0.9213362068965517
Escalizador = MinMaxScaler() solver= lbfgs , C= 1
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.9245689655172414
Escalizador = MinMaxScaler() solver= lbfgs , C= 5
matthews_corrcoef 0.6738432321543399
f1 = 0.8360655737704918
roc_auc = 0.9299568965517241
Escalizador = MinMaxScaler() solver= lbfgs , C= 10
matthews_corrcoef 0.6738432321543399
f1 = 0.8360655737704918
roc_auc = 0.9256465517241379
Escalizador = MinMaxScaler() solver= lbfgs , C= 50
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.927801724137931
Escalizador = MinMaxScaler() solver= lbfgs , C= 100
matthews_corrco

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

roc_auc = 0.9213362068965517
Escalizador = MinMaxScaler() solver= liblinear , C= 1
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.9245689655172414
Escalizador = MinMaxScaler() solver= liblinear , C= 5
matthews_corrcoef 0.6738432321543399
f1 = 0.8360655737704918
roc_auc = 0.9299568965517241
Escalizador = MinMaxScaler() solver= liblinear , C= 10
matthews_corrcoef 0.6738432321543399
f1 = 0.8360655737704918
roc_auc = 0.9256465517241379
Escalizador = MinMaxScaler() solver= liblinear , C= 50
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.927801724137931
Escalizador = MinMaxScaler() solver= liblinear , C= 100
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.9267241379310345
Escalizador = MinMaxScaler() solver= liblinear , C= 500
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.9267241379310345
Escalizador = MinMaxScaler() solver= sag , C= 0.01
matthews_corrcoef 0.47711642835569656
f1 = 0.721311

...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.927801724137931
Escalizador = StandardScaler() solver= liblinear , C= 100
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.927801724137931
Escalizador = StandardScaler() solver= liblinear , C= 500
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.927801724137931
Escalizador = StandardScaler() solver= sag , C= 0.01
matthews_corrcoef 0.7699801976524022
f1 = 0.8852459016393442
roc_auc = 0.9170258620689655
Escalizador = StandardScaler() solver= sag , C= 0.05
matthews_corrcoef 0.7699801976524022
f1 = 0.8852459016393442
roc_auc = 0.9245689655172413
Escalizador = StandardScaler() solver= sag , C= 0.1
matthews_corrcoef 0.7699801976524022
f1 = 0.8852459016393442
roc_auc = 0.9267241379310345
Escalizador = StandardScaler() solver= sag , C= 0.5
matthews_corrcoef 0.7370689655172413
f1 = 0.8688524590163934
roc_auc = 0.9256465517241379
Escalizador = StandardScaler() solver= sag , C=

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

matthews_corrcoef 0.7699801976524022
f1 = 0.8852459016393442
roc_auc = 0.9267241379310345
Escalizador = StandardScaler() solver= saga , C= 0.5
matthews_corrcoef 0.7370689655172413
f1 = 0.8688524590163934
roc_auc = 0.9256465517241379
Escalizador = StandardScaler() solver= saga , C= 1
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.9267241379310345
Escalizador = StandardScaler() solver= saga , C= 5
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.927801724137931
Escalizador = StandardScaler() solver= saga , C= 10
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.927801724137931
Escalizador = StandardScaler() solver= saga , C= 50
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.927801724137931
Escalizador = StandardScaler() solver= saga , C= 100
matthews_corrcoef 0.7050596119186784
f1 = 0.8524590163934426
roc_auc = 0.9267241379310345
Escalizador = StandardScaler() solver= saga , C= 500
matthews

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


###KNN

In [31]:
import numpy.matlib
import time
import matplotlib.pyplot as plt
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.neighbors import DistanceMetric
from sklearn.decomposition import PCA

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve,roc_auc_score

In [71]:
##Vectores para almacenar los hiperparámetros usados, y sus correspondientes puntajes

MCC_list=[]
F1_list=[]
AUC_list=[]

scaler_list=[]
nbrs_list=[]
weights_list=[]
metric_list=[]

for j in range(0,3):                    ##ITERACIÓN DEL ESCALIZADOR
  
  if j==0:
    scaler = MinMaxScaler()
    sca='MinMaxScaler()'
  elif j==1:
    scaler = RobustScaler()
    sca ='RobustScaler()'
  elif j==2:
    scaler = StandardScaler()
    sca='StandardScaler()'


  
  scaler.fit(X_train)
  X_train_1 = scaler.transform(X_train)
  X_test_1 = scaler.transform(X_test)

  pca = PCA(n_components=13)          ##13 COMPONENTES, LA DIMENSIÓN QUE MENOS VARIANZA EXPLICA, EXPLICA EL 2.6%, LAS DEJO TODAS
  pca.fit(X_train_1)
  X_train_1 = pca.transform(X_train_1)
  X_test_1 = pca.transform(X_test_1)
  print("Pesos de PCA:",pca.explained_variance_ratio_)
  #print("Componentes de PCA:",pca.components_)

  weights=[ 'uniform', 'distance']
  distancia=['euclidean','minkowski','manhattan','chebyshev']
  for n_nbrs in [1,3,5,7,9,11,13,15]:    ##ITERACION DE VECINOS
    for weight in range(0,2):            ##ITERACION DE PESOS
      for dist in range(0,4):            ##ITERACION DE DISTANCIA

            print('Escalizador =',sca,'n_neighbors=',n_nbrs,'|','weights =',weights[weight],'|','metric=',distancia[dist],'|')
            ##Almacenar los hiperparámetros en las listas
            scaler_list.append(scaler)
            nbrs_list.append(n_nbrs)
            weights_list.append(weights[weight])
            metric_list.append(distancia[dist])

            #Aplicar el modelo
            Knn = KNeighborsClassifier(n_neighbors=n_nbrs, weights= weights[weight] , metric=distancia[dist], metric_params=None, n_jobs=None)
            Knn.fit(X_train_1, y_train)
            y_test_predicted = Knn.predict(X_test_1)
            proba=Knn.predict_proba(X_test_1)##prob1
            y_test_scores = proba[:,1:].flatten()##prob2

            MCC = matthews_corrcoef(y_test, y_test_predicted)
            print("matthews_corrcoef", MCC)
            F1 = f1_score(y_test, y_test_predicted, average='micro')
            print("f1 =", F1)
            roc_auc=metrics.roc_auc_score(y_test, y_test_scores)
            print("roc_auc =", roc_auc)

            #Almacenar los scores en las listas
            MCC_list.append(MCC)
            F1_list.append(F1)
            AUC_list.append(roc_auc)

            
maxMCC=np.argmax(MCC_list)
maxF1=np.argmax(F1_list)
maxAUC=np.argmax(AUC_list)

print("Valor del mejor MCC = ",MCC_list[maxMCC])
print("Valor del mejor F1 = ",F1_list[maxF1])
print("Valor del mejor AUC_roc = ",AUC_list[maxAUC])

print("Parametros mejor MCC ",'Escalizador =',scaler_list[maxMCC],'n_neighbors=',nbrs_list[maxMCC],',','weights =',weights_list[maxMCC],',','metric=',metric_list[maxMCC])
print("Parametros mejor F1 ",'Escalizador =',scaler_list[maxF1],'n_neighbors=',nbrs_list[maxF1],',','weights =',weights_list[maxF1],',','metric=',metric_list[maxF1])
print("Parametros mejor AUC_roc ",'Escalizador =',scaler_list[maxAUC],'n_neighbors=',nbrs_list[maxAUC],',','weights =',weights_list[maxAUC],',','metric=',metric_list[maxAUC])

Pesos de PCA: [0.27173097 0.19243203 0.11711389 0.08507425 0.08130784 0.06387459
 0.05517117 0.03314911 0.03074089 0.02484146 0.01832923 0.01605866
 0.0101759 ]
Escalizador = MinMaxScaler() n_neighbors= 1 | weights = uniform | metric= euclidean |
matthews_corrcoef 0.6213573271500317
f1 = 0.8032786885245902
roc_auc = 0.8076508620689655
Escalizador = MinMaxScaler() n_neighbors= 1 | weights = uniform | metric= minkowski |
matthews_corrcoef 0.6213573271500317
f1 = 0.8032786885245902
roc_auc = 0.8076508620689655
Escalizador = MinMaxScaler() n_neighbors= 1 | weights = uniform | metric= manhattan |
matthews_corrcoef 0.6500637093654297
f1 = 0.819672131147541
roc_auc = 0.8232758620689656
Escalizador = MinMaxScaler() n_neighbors= 1 | weights = uniform | metric= chebyshev |
matthews_corrcoef 0.6598148251201058
f1 = 0.819672131147541
roc_auc = 0.8248922413793103
Escalizador = MinMaxScaler() n_neighbors= 1 | weights = distance | metric= euclidean |
matthews_corrcoef 0.6213573271500317
f1 = 0.803278

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

matthews_corrcoef 0.7451421267603893
f1 = 0.8688524590163934
roc_auc = 0.9073275862068966
Escalizador = MinMaxScaler() n_neighbors= 3 | weights = distance | metric= minkowski |
matthews_corrcoef 0.7451421267603893
f1 = 0.8688524590163934
roc_auc = 0.9073275862068966
Escalizador = MinMaxScaler() n_neighbors= 3 | weights = distance | metric= manhattan |
matthews_corrcoef 0.6738432321543399
f1 = 0.8360655737704918
roc_auc = 0.8992456896551724
Escalizador = MinMaxScaler() n_neighbors= 3 | weights = distance | metric= chebyshev |
matthews_corrcoef 0.7161535198175818
f1 = 0.8524590163934426
roc_auc = 0.855603448275862
Escalizador = MinMaxScaler() n_neighbors= 5 | weights = uniform | metric= euclidean |
matthews_corrcoef 0.6738432321543399
f1 = 0.8360655737704918
roc_auc = 0.8798491379310345
Escalizador = MinMaxScaler() n_neighbors= 5 | weights = uniform | metric= minkowski |
matthews_corrcoef 0.6738432321543399
f1 = 0.8360655737704918
roc_auc = 0.8798491379310345
Escalizador = MinMaxScaler()

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

matthews_corrcoef 0.7747844827586207
f1 = 0.8852459016393442
roc_auc = 0.9040948275862069
Escalizador = MinMaxScaler() n_neighbors= 5 | weights = distance | metric= chebyshev |
matthews_corrcoef 0.7161535198175818
f1 = 0.8524590163934426
roc_auc = 0.8900862068965517
Escalizador = MinMaxScaler() n_neighbors= 7 | weights = uniform | metric= euclidean |
matthews_corrcoef 0.6598148251201058
f1 = 0.819672131147541
roc_auc = 0.8841594827586207
Escalizador = MinMaxScaler() n_neighbors= 7 | weights = uniform | metric= minkowski |
matthews_corrcoef 0.6598148251201058
f1 = 0.819672131147541
roc_auc = 0.8841594827586207
Escalizador = MinMaxScaler() n_neighbors= 7 | weights = uniform | metric= manhattan |
matthews_corrcoef 0.6500637093654297
f1 = 0.819672131147541
roc_auc = 0.8987068965517242
Escalizador = MinMaxScaler() n_neighbors= 7 | weights = uniform | metric= chebyshev |
matthews_corrcoef 0.6213573271500317
f1 = 0.8032786885245902
roc_auc = 0.8766163793103449
Escalizador = MinMaxScaler() n_n

...

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

matthews_corrcoef 0.7747844827586207
f1 = 0.8852459016393442
roc_auc = 0.9181034482758621
Escalizador = StandardScaler() n_neighbors= 9 | weights = distance | metric= chebyshev |
matthews_corrcoef 0.7395052723482932
f1 = 0.8688524590163934
roc_auc = 0.9407327586206897
Escalizador = StandardScaler() n_neighbors= 11 | weights = uniform | metric= euclidean |
matthews_corrcoef 0.7707216521126317
f1 = 0.8852459016393442
roc_auc = 0.9143318965517242
Escalizador = StandardScaler() n_neighbors= 11 | weights = uniform | metric= minkowski |
matthews_corrcoef 0.7707216521126317
f1 = 0.8852459016393442
roc_auc = 0.9143318965517242
Escalizador = StandardScaler() n_neighbors= 11 | weights = uniform | metric= manhattan |
matthews_corrcoef 0.7707216521126317
f1 = 0.8852459016393442
roc_auc = 0.9272629310344828
Escalizador = StandardScaler() n_neighbors= 11 | weights = uniform | metric= chebyshev |
matthews_corrcoef 0.7699801976524022
f1 = 0.8852459016393442
roc_auc = 0.9358836206896551
Escalizador = S

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

matthews_corrcoef 0.7699801976524022
f1 = 0.8852459016393442
roc_auc = 0.9078663793103448
Escalizador = StandardScaler() n_neighbors= 13 | weights = uniform | metric= manhattan |
matthews_corrcoef 0.802801724137931
f1 = 0.9016393442622952
roc_auc = 0.9143318965517242
Escalizador = StandardScaler() n_neighbors= 13 | weights = uniform | metric= chebyshev |
matthews_corrcoef 0.7699801976524022
f1 = 0.8852459016393442
roc_auc = 0.9261853448275862
Escalizador = StandardScaler() n_neighbors= 13 | weights = distance | metric= euclidean |
matthews_corrcoef 0.7699801976524022
f1 = 0.8852459016393442
roc_auc = 0.9137931034482759
Escalizador = StandardScaler() n_neighbors= 13 | weights = distance | metric= minkowski |
matthews_corrcoef 0.7699801976524022
f1 = 0.8852459016393442
roc_auc = 0.9137931034482759
Escalizador = StandardScaler() n_neighbors= 13 | weights = distance | metric= manhattan |
matthews_corrcoef 0.7707216521126317
f1 = 0.8852459016393442
roc_auc = 0.9170258620689655
Escalizador =

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

### ANN

In [40]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as kr
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve,roc_auc_score

In [59]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_1 = scaler.transform(X_train)
X_test_1 = scaler.transform(X_test)

pca = PCA(n_components=13)          ##13 COMPONENTES, LA DIMENSIÓN QUE MENOS VARIANZA EXPLICA, EXPLICA EL 2.6%, LAS DEJO TODAS
pca.fit(X_train_1)
X_train_1 = pca.transform(X_train_1)
X_test_1 = pca.transform(X_test_1)
print("Pesos de PCA:",pca.explained_variance_ratio_)
#print("Componentes de PCA:",pca.components_)

Pesos de PCA: [0.2072575  0.12434085 0.09450576 0.08846415 0.08026789 0.07561969
 0.0678882  0.06005231 0.05322755 0.04714201 0.04072549 0.03340773
 0.02710085]


In [ ]:
nn_model = Sequential()
nn_model.add(Dense(9, input_dim=X_train_1.shape[1] , activation='relu'))
nn_model.add(Dense(9,activation='relu'))
nn_model.add(Dense(1,activation='sigmoid'))

nn_model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1)
history = nn_model.fit(X_train_1, y_train, epochs=100, batch_size=1, verbose=1, callbacks=[early_stop], shuffle=False)


In [65]:
nn_model.evaluate(X_test_1, y_test)
y_pred_test_nn  = nn_model.predict(X_test_1)

nn_pred = []
for i in y_pred_test_nn:
  if i > 0.5:
    nn_pred.append(1)
  else:
    nn_pred.append(0)

#predicciones= pd.DataFrame(nn_pred)
print("Las predicciones realizadas son las siguientes:\n",np.transpose(nn_pred))
print("Las etiquetas contra las que compararemos\n",np.transpose(y_test))

2/2 [==============================] - 0s 4ms/step - loss: 0.1525
Las predicciones realizadas son las siguientes:
 [0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 0 0 0]
Las etiquetas contra las que compararemos
 [[0 0 1 0 1 1 1 0 0 1 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 0
  1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0]]


In [67]:
from sklearn.metrics import classification_report as cr
print(cr(y_test, nn_pred))

              precision    recall  f1-score   support

           0       0.82      0.79      0.81        29
           1       0.82      0.84      0.83        32

    accuracy                           0.82        61
   macro avg       0.82      0.82      0.82        61
weighted avg       0.82      0.82      0.82        61



### Conclusiones
El tercer clasificador mejor implementado fue ANN, que deja mucho margen de mejora ya que tiene varios hiperparámetros que se pueden cambiar. Con esto, se logró una puntuación mínima de nuestros datos de prueba de 0,79, pero es posible una mejora de este resultado, como se dijo antes. A medida que se puedan agregar más capas ocultas, aumente la cantidad de neuronas y cambie las funciones de activación de la misma manera. Todo esto nos da más grados de libertad para mejorar nuestra red neuronal artificial.

El clasificador con la mejor puntuación fue KNN, que dio una puntuación de 0,94 de AUC para los datos de la prueba.

Los resultados de los distintos métodos, al aplicar PCA mejoraron considerablemente, aún cuando no reduje la dimensionalidad del problema, esto también fue porque a mi criterio no hubo ningina dimensión que explicara considarablemente menos porcentaje de la varianza respecto a otras dimensiones. De esto concluimos que las características presentes en el conjunto de datos tienen una influencia significativa en la salida de nuestros algoritmos de aprendizaje supervisado implementados, esto puede deberse a que las características no son combinaciones lineales entre ellas.